#  Projeto de Machine Learning - Random Forest Regressor

##  Visão Geral
Este notebook tem como objetivo desenvolver um **modelo de Machine Learning - Regressão Linear ** utilizando o dataset deste projeto sendo dados ficticios e gerados de maneira aleatória.  
O foco principal é aplicar técnicas de **pré-processamento, seleção de variáveis e modelagem preditiva**, avaliando o desempenho do modelo por meio de métricas adequadas.

---
##  Sobre o Dataset

O dataset contém informações relacionadas a ** informações de clientes que terão maiores impactos na descisão de limite de credio **.  
Os dados foram obtidos a partir do código **[origem dos dados]**

Cada linha do dataset representa **um cliente**, enquanto as colunas representam **variáveis explicativas (features)** e a **variável alvo (limite_credito)** utilizada no treinamento do modelo.

---

##  Objetivo do Projeto

O objetivo deste projeto é **construir um modelo de Machine Learning utilizando modelo Random Forest Regressor capaz de prever**: 

> **Qual limite de crédito do cliente.**

Com isso, busca-se identificar os principais fatores que influenciam a variável alvo


---

##  Estrutura dos Dados

O dataset é composto por variáveis do tipo:
- **Numéricas** (ex: idade, valor, renda, tempo de relacionamento, score de credito, limite de credito)

---

##  Etapas do Projeto

As principais etapas deste projeto incluem:

1. Importação das bibliotecas
2. Leitura e entendimento do dataset
3. Tratamento de dados (valores nulos, outliers e inconsistências)
4. Separação entre variáveis independentes (X) e variável alvo (y)
5. Divisão dos dados em treino e teste
6. Treinamento de modelos de Machine Learning
7. Avaliação do desempenho dos modelos
8. Interpretação dos resultados

---

##  Métricas de Avaliação

As métricas utilizadas para avaliar o modelo dependem do tipo de problema:

- **Classificação**: Accuracy, Precision, Recall, F1-score, ROC-AUC
- **Regressão**: RMSE, MAE, R²

---

##  Próximos Passos

Ao final do notebook, serão apresentadas:
- As métricas de desempenho dos modelos
- A importância das variáveis
- Conclusões e possíveis melhorias futuras

---


## Manipulação de dados e Análise Exploratória 

In [1]:
import pandas as pd
import numpy as np


# criando dataframe com pandas do arquivo
df = pd.read_csv("dataset.csv")


# exibir  quantidade de linhas e colunas 
df.shape

(100000, 5)

In [2]:
df.describe()
# descrição geral do dataset (média, desvio padrão...)
## Identificado valores negativos para limite de credito de alguns clientes
## Para todos valores negativos da variavel limite_credito vamos incluir o "0"


,renda_mensal,idade,score_credito,tempo_relacionamento,limite_credito
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,5034.433664,43.578750,74.490090,6.992950,5383.307058
std,1931.627766,14.992038,14.461486,4.316689,2525.657695
min,1500.000000,18.000000,50.000000,0.000000,-2860.890000
25%,3651.012500,31.000000,62.000000,3.000000,3462.600000
50%,5005.300000,44.000000,74.000000,7.000000,5373.085000
75%,6353.832500,57.000000,87.000000,11.000000,7301.605000
max,13958.170000,69.000000,99.000000,14.000000,13840.970000


In [3]:
## Alterando valores negativos em limite de credito para 0
df_1 = df.copy()
df_1.loc[df_1["limite_credito"] < 0, "limite_credito"] = 0
df_1.describe()


,renda_mensal,idade,score_credito,tempo_relacionamento,limite_credito
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,5034.433664,43.578750,74.490090,6.992950,5387.622959
std,1931.627766,14.992038,14.461486,4.316689,2515.633749
min,1500.000000,18.000000,50.000000,0.000000,0.000000
25%,3651.012500,31.000000,62.000000,3.000000,3462.600000
50%,5005.300000,44.000000,74.000000,7.000000,5373.085000
75%,6353.832500,57.000000,87.000000,11.000000,7301.605000
max,13958.170000,69.000000,99.000000,14.000000,13840.970000


In [4]:
df.info()
# informações geral do dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   renda_mensal          100000 non-null  float64
 1   idade                 100000 non-null  int64  
 2   score_credito         100000 non-null  int64  
 3   tempo_relacionamento  100000 non-null  int64  
 4   limite_credito        100000 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 3.8 MB


In [5]:
# identificar correlação entre as variaveis
df.corr()
# Quanto maior o tempo de relacionamento, maior tende a ser o limite de crédito.

,renda_mensal,idade,score_credito,tempo_relacionamento,limite_credito
renda_mensal,1.000000,-0.004349,-0.006424,0.002605,0.308342
idade,-0.004349,1.000000,0.003633,-0.001555,-0.123824
score_credito,-0.006424,0.003633,1.000000,-0.002130,0.051439
tempo_relacionamento,0.002605,-0.001555,-0.002130,1.000000,0.855149
limite_credito,0.308342,-0.123824,0.051439,0.855149,1.000000


In [6]:
# analise de correlação entre a variavel target "limite de credito"
df.corr()["limite_credito"].sort_values(ascending=False)


limite_credito          1.000000
tempo_relacionamento    0.855149
renda_mensal            0.308342
score_credito           0.051439
idade                  -0.123824
Name: limite_credito, dtype: float64

In [7]:
df.quantile([0.01, 0.99])
#identificação de outliers
#A análise dos quantis de 1% e 99% indica que a maior parte dos clientes apresenta renda mensal entre R$ 1.500 e R$ 9.645,
#idade entre 18 e 69 anos 
#limite de crédito concentrado entre aproximadamente R$ 116 e R$ 10.677. 


,renda_mensal,idade,score_credito,tempo_relacionamento,limite_credito
0.01,1500.0000,18.0,50.0,0.0,116.5297
0.99,9645.2292,69.0,99.0,14.0,10677.8490


### Conclusões da Análise Exploratória

- Dataset com 100 Mil observações (linhas) e 5 variáveis (colunas)
- O dataset não apresenta valores nulos.
- Identifcado valores negativos para limite de crédito e realizado alteração para **0**
- A variável e 'tempo_relacinamento'  'renda_mensal' apresenta maiores correlação positiva com o limite de crédito.
- O score de crédito também exerce influência relevante na variável alvo.



--

##  Preparar para Treinar o Modelo

#### Separação dataset entre variáveis independentes (X) e target (alvo) (y)

In [8]:
# import bibliotica que separar os dados de treino e teste
from sklearn.model_selection import train_test_split

# X contém as variáveis que o modelo usará para aprender
# y contém o valor que queremos prever (limite de crédito)
X = df.drop("limite_credito", axis=1)
y = df["limite_credito"]

#### Separando entre  Treino e Teste

In [9]:
# # Separação dos dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [10]:
# Visualizando os dados separados 70% treino e 30% teste
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(70000, 4)
(30000, 4)
(70000,)
(30000,)


#### Criando o modelo **RandomForestRegressor**

In [11]:
# importando a biblioteca e cirando o modelo **RandomForestRegressor**
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [12]:
y_pred = model.predict(X_test)


In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")




MAE: 861.64
RMSE: 1079.28
R²: 0.82


#### AVALIANDO RESULTADOS 
MAE = 861,64 → em média, as previsões do modelo erram cerca de 862 unidades em relação aos valores reais.

RMSE = 1.079,28 → indica que erros maiores existem, mas ainda estão dentro de um intervalo aceitável para o problema.

R² = 0,82 → o modelo consegue explicar 82% da variação dos dados, o que representa um bom desempenho preditivo.

#### Avaliando a importancia de cada váriavel na decisão do Modelo

In [13]:
importancias = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print(importancias)

tempo_relacionamento    0.731558
renda_mensal            0.163876
idade                   0.054579
score_credito           0.049987
dtype: float64
